In [1]:
import re
import numpy as np

In [2]:
import re

def checknum(word):
    
    pattern = re.compile('[0-9]+')
    result = pattern.findall(word)
#     print(result)
    return len(result)
    


def countcapital(word):
    count = 0
    for i in word:
        if i.isupper():
            count += 1
    return count
# countcapital("HEere")
checknum('1jhkj432hj3h')


3

In [3]:
def checknum(word):
    
    pattern = re.compile('[0-9]+')
    result = pattern.findall(word)
#     print(result)
    return len(result)
    
# print(checknum('1jhkj42hjh'))


def getfeatures(word):
    # 定义向量features
    features = []
    
    # 判断是否全为字母
    features.append(word.isalpha()+0)
#     print("是否全为字母\t",word.isalpha()+0)
    
    # 判断字母是否全是大写
    features.append(word.isupper()+0)
#     print("是否所有字母都是大写\t",word.isupper()+0)
    
    # 判断是否全部是数字
    features.append(word.isdigit()+0)
#     print("是否全部是数字\t",word.isdigit()+0)
    
    #判断有多少数字
    features.append(checknum(word)+0)
#     print("有多少数字\t",checknum(word)+0)
    
    # 判断是否全部为字母或数字组成
    features.append(word.isalnum()+0)
#     print("是否全为字母或者数字\t",word.isalnum()+0)
    
    # 判断首字母是否大写
    features.append(word.istitle()+0)
#     print("首字母是否大写\t",word.istitle()+0)
    # 返回结果
#     return np.array(features)

#    大写字母个数
    c = countcapital(word)
    features.append(c)
#   单词长度
    l = len(word)
    features.append(l)
#    以n结尾
#    字母占比
    
#    数字占比
    features.append(checknum(word)/l+0)
#    大写字母占比
    features.append(c/l)

    return features
getfeatures('djflskjafiLLLIIJ12434')
# checknum('122l22')

[0, 0, 0, 1, 1, 0, 6, 21, 0.047619047619047616, 0.2857142857142857]

In [4]:
def getag(tag):
    tag = tag.replace("\n","")
    if tag == "TAG":
        return 0
    if tag =="GENE1":
        return 1
    if tag == "GENE2":
        return 2

In [5]:
# 打开文件，第一个参数为文件的路径，第二个参 数为读写权限，r代表只允许读
f = open('data/train.tag', 'r')

X = []
Y = []


# 读取文件的每一行
for line in f.readlines():

    # 若该行包含“_”，则证明该行不是序号，是数据
    if "_" in line:
        # print(line)

        # 将每一行按照空格分隔，则会返回一个列表，列表内的元素为 单词_标签 的形式
        line = line.split(" ")
#         print(line)

        for word_tag in line:
            word_tag = word_tag.split("_")
            word = word_tag[0]
            tag = word_tag[1]
#             print(getfeatures(word))
            
            # 后面可以加一些对word和tag的操作
            X.append(getfeatures(word))
            Y.append(getag(tag))
            
X = np.array(X)
Y = np.array(Y)

print(X)
print(Y)


[[ 1.   0.   0.  ... 10.   0.   0.1]
 [ 1.   0.   0.  ...  4.   0.   0. ]
 [ 1.   0.   0.  ...  7.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  1.   0.   0. ]
 [ 1.   0.   0.  ...  2.   0.   0.5]
 [ 0.   0.   0.  ...  1.   0.   0. ]]
[0 0 0 ... 0 0 0]


In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.1)
print(x_train)
print(x_test)
print(y_train)
print(y_test)

[[1.  0.  0.  ... 2.  0.  0.5]
 [1.  0.  0.  ... 2.  0.  0. ]
 [1.  0.  0.  ... 3.  0.  0. ]
 ...
 [1.  0.  0.  ... 2.  0.  0.5]
 [1.  0.  0.  ... 2.  0.  0. ]
 [1.  0.  0.  ... 2.  0.  0. ]]
[[1. 0. 0. ... 7. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 4. 0. 0.]
 ...
 [1. 0. 0. ... 6. 0. 0.]
 [1. 0. 0. ... 8. 0. 0.]
 [0. 0. 1. ... 1. 1. 0.]]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [7]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2',class_weight='balanced')
model.fit(x_train,y_train)
train_score = model.score(x_train,y_train)
test_score = model.score(x_test,y_test)
print('train score: {train_score:.6f}; test_score:{test_score:.6f}'
      .format(train_score=train_score,
             test_score=test_score))

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


train score: 0.875705; test_score:0.879573


In [8]:
y_pred_proba = model.predict_proba(x_test)
i=0
while i < 10000:
    
    if y_test[i] != 0:
        print(y_pred_proba[i],y_test[i])
    i += 1

[0.72524699 0.13742013 0.13733288] 2
[0.56337059 0.21518127 0.22144814] 1
[0.29221391 0.34892482 0.35886127] 2
[0.17209279 0.41787256 0.41003465] 1
[0.39004718 0.30233156 0.30762126] 2
[0.66140622 0.1695274  0.16906638] 1
[0.66140622 0.1695274  0.16906638] 1
[0.68348584 0.15841988 0.15809428] 1
[0.11718298 0.43815928 0.44465774] 2
[0.21536211 0.39480765 0.38983025] 2
[0.29221391 0.34892482 0.35886127] 1
[0.68348584 0.15841988 0.15809428] 1
[0.20318829 0.39990379 0.39690792] 1
[0.61520547 0.19277728 0.19201726] 2
[0.68348584 0.15841988 0.15809428] 2
[0.68348584 0.15841988 0.15809428] 2
[0.21536211 0.39480765 0.38983025] 1
[0.66140622 0.1695274  0.16906638] 1
[0.62792547 0.1847141  0.18736043] 2
[0.62792547 0.1847141  0.18736043] 2
[0.05608233 0.47586939 0.46804828] 1
[0.62792547 0.1847141  0.18736043] 2
[0.63165409 0.17660113 0.19174478] 2
[0.56332715 0.22019369 0.21647916] 1
[0.04582769 0.47937023 0.47480208] 1
[0.18655204 0.41178614 0.40166182] 1
[0.31247208 0.33605042 0.3514775 ] 2
[

[0.68348584 0.15841988 0.15809428] 2
[0.72524699 0.13742013 0.13733288] 2
[0.39004718 0.30233156 0.30762126] 1
[0.74482141 0.12758189 0.1275967 ] 2
[0.29221391 0.34892482 0.35886127] 1
[0.09046994 0.45353914 0.45599092] 2
[0.21536211 0.39480765 0.38983025] 2
[0.66140622 0.1695274  0.16906638] 2
[0.18655204 0.41178614 0.40166182] 2
[0.66714143 0.16324852 0.16961005] 2
[0.14255967 0.43251005 0.42493029] 1
[0.49228703 0.25465523 0.25305773] 2
[0.24133631 0.38396032 0.37470337] 2
[0.78118015 0.1093177  0.10950214] 2
[0.56337059 0.21518127 0.22144814] 2
[0.70478596 0.14770743 0.14750661] 1
[0.31247208 0.33605042 0.3514775 ] 2
[0.62792547 0.1847141  0.18736043] 2
[0.62792547 0.1847141  0.18736043] 1
[0.14255967 0.43251005 0.42493029] 1
[0.51426754 0.2450817  0.24065076] 2
[0.09874018 0.45088239 0.45037743] 2
[0.66140622 0.1695274  0.16906638] 2
[0.56337059 0.21518127 0.22144814] 1
[0.59125984 0.20483056 0.2039096 ] 1
[0.10371122 0.45274808 0.4435407 ] 2
[0.10260643 0.4527091  0.44468447] 1
[

In [9]:
y_pred = model.predict(x_test)

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("accuracy score:",accuracy_score(y_test,y_pred ))
print("precision_score",precision_score(y_test,y_pred, average='macro'))
print("recall_score:",recall_score(y_test,y_pred, average='macro'))
print("f1_score:",f1_score(y_test,y_pred, average='macro'))


print("precision_score",precision_score(y_test,y_pred, average='micro'))
print("recall_score:",recall_score(y_test,y_pred, average='micro'))
print("f1_score:",f1_score(y_test,y_pred, average='micro'))

print("precision_score",precision_score(y_test,y_pred, average='weighted'))
print("recall_score:",recall_score(y_test,y_pred, average='weighted'))
print("f1_score:",f1_score(y_test,y_pred, average='weighted'))

print("precision_score",precision_score(y_test,y_pred, average=None))
print("recall_score:",recall_score(y_test,y_pred, average=None))
print("f1_score:",f1_score(y_test,y_pred, average=None))

accuracy score: 0.8795728612611149
precision_score 0.4568294484812519
recall_score: 0.43998572574164885
f1_score: 0.4445723878301158
precision_score 0.8795728612611149
recall_score: 0.8795728612611149
f1_score: 0.8795728612611149
precision_score 0.8631150243738482
recall_score: 0.8795728612611149
f1_score: 0.8706683098001498
precision_score [0.93248408 0.26804979 0.16995448]
recall_score: [0.95466811 0.27257384 0.09271523]
f1_score: [0.9434457  0.27029289 0.11997858]
